# Scraping the two news sites with most coments, 20Min & Watson

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import time
import twython

datestring = time.strftime("%Y-%h-%d")
datestring

'2016-Jul-08'

In [2]:
api_key = ""
api_secret = ""

access_token = ""
token_secret = ""

In [3]:
twitter = twython.Twython(api_key, api_secret, access_token, token_secret)

In [4]:
# Grab the 20Min & Watson Homepages
response = requests.get("http://www.20min.ch")
response_watson = requests.get("http://www.watson.ch/")
response_Tagi = requests.get("http://www.tagesanzeiger.ch")

In [5]:
# Feed them into BeautifulSoup
_20Min = BeautifulSoup(response.text, 'html.parser')
Watson = BeautifulSoup(response_watson.text, 'html.parser')
Tagi = BeautifulSoup(response_Tagi.text, 'html.parser')

In [6]:
stories = _20Min.find_all("div", {'id': 'content'})
stories_watson = Watson.find_all('div', {'class': 'teaser teaser_type_default     font_m  combo_bg combo_fg'})
Tagi_stories = Tagi.find_all("div", {'class': 'featureStory standard'})
Tagi_kommentare = Tagi.find_all("div", {'class': 'articleStory valued'})

In [7]:
#finding everything outside of the loop
teaser_stories = stories[0].find_all('div', {'class': "teaser "})

#starting the dictionary:
all_stories = []

#The Loop
for story in teaser_stories:
    
    #Getting the comments
    comments = story.find('div', {'class': "teaser_media front clearfix"})
    if comments:
        try:
            comment = comments.find('span', {'class': 'icon_talkback'}).next_sibling
            #Just using the number of comments
            comment = comment[:4]
            #A regular expression to make sure to get articles with more of 999 comments
            if re.search(r"\D$", comment):
                comment = int(comment[:3])
            else:
                comment = int(comment)
        except:
            comment = 1
    
    #Getting the the title
    title_div = story.find('div', {'class': "teaser_title front"})
    title_string = title_div.find('h2').string
    if re.search(r"\x96", title_string):
        title_string.replace("\x96", "")
        
    
    #Getting the HREF
    href = story.find('h3')
    href = href.find('a', href=True)
    href = href['href']
    
    #Dealing with non-full links
    if re.search(r"^http:", href):
        href = href
    else:
        href = 'http://www.20min.ch' + href
    
    #Creating the dictionary:
    dictionary = {'Site': '20Min', 'Titel': title_string, 'Kommentare': int(comment), 'Link': href, 'Date': datestring}
    
    #List of dictionaries:
    all_stories.append(dictionary)

In [8]:
#starting the dictionary:
all_stories_watson = []

#looping through titles, comments, links
for story in stories_watson:
    
    #getting HREF
    href = story.find('a', {'class': 'storylink combo_bd'})
    try:
        href = href['href']
        href = ('http://watson.ch' + href)
    except:
        href == None
        
    #Getting the story
    title = story.find('h2')
    if title == None:
        title = None
    else:
        title = title.string
    
    #Getting the Comments
    comments = story.find('a', {'class': 'comments'})
    if comments == None:
        comments = 0
    else:
        comments = story.find('img', {'src': '/media/img/main/icons/icon_comments.png'}).span.text
        comments = int(comments)
    
    #creating the dictionary
    dictionary = {'Site': 'Watson', 'Titel': title, 'Kommentare': int(comments), 'Link': href, 'Date': datestring}
    
    #creating the list
    all_stories_watson.append(dictionary)

In [9]:
Tagi_Dict = []

#looping through all the stories
for x in Tagi_stories:
    
    #Getting each headline and extracting only the text
    try:
        title = x.find('h3').get_text().strip()
    except:
        title == None

    #Getting the byline
    #time = title.find('p', {'class' : 'storyInfo'})
    
    #Getting time
    #time = x.find('p', {'class' : 'storyInfo'})
    #time = time.find('time', {'class' : 'time'})
    
    #Getting Kommentare
    if x.find('a', {'title' : 'Kommentare'}) is None:
        kommentare = 0
    else:
        kommentare = x.find('a', {'title' : 'Kommentare'}).string
        #kommentare = kommentare.replace(u'\xa0', ' ')
        kommentare = int(re.findall(r"\d+", kommentare)[0])
        
    #Getting HREF
    try:
        href = x.find('h3')
        href = href.find('a').get('href')
        href = 'http://www.tagesanzeiger.ch' + href
    except:
        href == None
        
    #Creating dictionary with Tagi Headlines, time
    title = { 'Site': 'Tagi', 'Titel': title, 'Kommentare': kommentare, 'Link': href, 'Date': datestring}
    
    Tagi_Dict.append(title)

In [10]:
Tagi_Kommentare_Dict = []

for x in Tagi_kommentare:
    
    #Getting each headline and extracting only the text
    try:
        title = x.find('h4').get_text().strip()
    except:
        title == None
    
    #Getting Kommentare
    if x.find('span', {'class' : 'icon icon-comment'}) is None:
        kommentare = 0
    else:
        kommentare = x.find('span', {'class' : 'icon icon-comment'}).next_sibling
        kommentare = kommentare[:4]
        #A regular expression to make sure to get articles with more of 999 comments
        if re.search(r"\D$", kommentare):
            kommentare = int(kommentare[:3])
        else:
            kommentare = int(kommentare)
    #Getting HREF
    try:
        href = x.find('h4')
        href = href.find('a').get('href')
        href = 'http://www.tagesanzeiger.ch' + href
    except:
        href == None
        
    #Creating dictionary with Tagi Headlines, time
    Kommentare = { 'Site': 'Tagi', 'Titel': title, 'Kommentare': kommentare, 'Link': href, 'Date': datestring}
    
    Tagi_Kommentare_Dict.append(Kommentare)

In [11]:
#adding the two dictionaries
all_stories.extend(all_stories_watson)
all_stories.extend(Tagi_Dict)
all_stories.extend(Tagi_Kommentare_Dict)

In [12]:
#sorting the dictionary according to the value of the keys
import operator
all_stories.sort(key=operator.itemgetter('Kommentare'), reverse=True)

In [14]:
all_stories[0]

{'Date': '2016-Jul-08',
 'Kommentare': 572,
 'Link': 'http://www.20min.ch/schweiz/energy-challenge/story/Lohnt-es-sich--den-Motor-am-Rotlicht-abzuschalten--21177214',
 'Site': '20Min',
 'Titel': 'Lohnt es sich, den Motor am Rotlicht abzuschalten?'}

In [15]:
all_stories[0]['Titel']

'Lohnt es sich, den Motor am Rotlicht abzuschalten?'

In [16]:
len(all_stories)

181

In [17]:
Sum = 0
for item in all_stories:
    Sum = Sum + item['Kommentare']

In [18]:
Sum

5014

In [19]:
Kommentar_Schnitt = round(Sum / len(all_stories))

In [20]:
Count = all_stories[0]['Kommentare']
#if Count > 1100:
#    Comment_Count = "Lesen. Dazu hat Krethi und Plethi eine Meinung."
#elif Count > 900:
#    Comment_Count = "Party-Talk. Wirklich an jeder Party."
#elif Count > 700:
#    Comment_Count = "Da haben am Stammtisch einige etwas dazu zu sagen."
#elif Count > 500:
#    Comment_Count = "Das bewegt einge. Am richtig viele sind das noch nicht."
#elif Count > 0:
#    Comment_Count = "Da schlafen einige fleissige Kommentarschreiben wohl noch."

In [21]:
#sentence = 'Kommentare: {}. Quelle: {}. {} {}'.format(all_stories[0]['Kommentare'], all_stories[0]['Site'], Comment_Count, all_stories[0]['Link'])

In [22]:
sentence = 'Die heisseste Front-Story läuft auf {} und hat {} Kommentare. Der Schnitt liegt bei {} Reaktionen pro Story. {}'.format(all_stories[0]['Site'], all_stories[0]['Kommentare'], Kommentar_Schnitt, all_stories[0]['Link']) 

In [23]:
print(sentence)

Die heisseste Front-Story läuft auf 20Min und hat 572 Kommentare. Der Schnitt liegt bei 28 Reaktionen pro Story. http://www.20min.ch/schweiz/energy-challenge/story/Lohnt-es-sich--den-Motor-am-Rotlicht-abzuschalten--21177214


In [24]:
#twitter = twython.Twython(api_key, api_secret)
#auth = twitter.get_authentication_tokens()
#print("Log into Twitter as the user you want to authorize and visit this URL:")
#print("\t" + auth['auth_url'])

In [25]:
#pin = "XXXXXX"
#
#twitter = twython.Twython(api_key, api_secret, auth['oauth_token'], auth['oauth_token_secret'])
#tokens = twitter.get_authorized_tokens(pin)
#
#new_access_token = tokens['oauth_token']
#new_token_secret = tokens['oauth_token_secret']
#print("your access token:", new_access_token)
#print("your token secret:", new_token_secret)

In [26]:
twitter.update_status(status=sentence)

{'contributors': None,
 'coordinates': None,
 'created_at': 'Fri Jul 08 16:22:04 +0000 2016',
 'entities': {'hashtags': [],
  'symbols': [],
  'urls': [{'display_url': '20min.ch/schweiz/energy…',
    'expanded_url': 'http://www.20min.ch/schweiz/energy-challenge/story/Lohnt-es-sich--den-Motor-am-Rotlicht-abzuschalten--21177214',
    'indices': [113, 136],
    'url': 'https://t.co/NbccGRlOcA'}],
  'user_mentions': []},
 'favorite_count': 0,
 'favorited': False,
 'geo': None,
 'id': 751451293642817537,
 'id_str': '751451293642817537',
 'in_reply_to_screen_name': None,
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'is_quote_status': False,
 'lang': 'de',
 'place': None,
 'possibly_sensitive': False,
 'retweet_count': 0,
 'retweeted': False,
 'source': '<a href="http://www.barnabyskinner.com/" rel="nofollow">First_App_data_and_databases</a>',
 'text': 'Die heisseste Front-Story läuft auf 20Min und hat 572